# [Travelbrag](http://travelbrag.ca)
Andrea & Brett's travelogue.

In [42]:
import datetime
import pandas

cities = pandas.read_csv("cities.csv")
travelogue = pandas.read_csv("travelogue.csv", parse_dates=["arrived", "departed"])
today = datetime.date.today()

# What was our last vacation?
"Vacation" is classified as the last time Andrea and I travelled together.

In [47]:
vacations = travelogue[travelogue["travellers"] == "Andrea & Brett"]
last_vacation = vacations.sort_values(by="departed").tail(1).iloc[0]

print("Our last vacation was {}, {} days ago.".format(last_vacation["purpose"],
                                                     (today - last_vacation["departed"].date()).days))

Our last vacation was PyCon, 127 days ago.


# Where have we been in the last 12 months?

In [43]:
import datetime

year_ago = today - datetime.timedelta(days=365)
travelogue[travelogue["arrived"] >= year_ago].sort_values(by="arrived")

,year,purpose,travellers,city,country,arrived,departed,photos,comment
147,2015,PyCon CA/PyData NYC,Andrea & Brett,Toronto,CAN,2015-11-06,2015-11-08,NaN,NaN
148,2015,PyCon CA/PyData NYC,Andrea & Brett,New York,USA,2015-11-08,2015-11-11,NaN,NaN
149,2015,PyCon CA/PyData NYC,Andrea & Brett,Union City,USA,2015-11-11,2015-11-15,NaN,NaN
150,2015,PyCon CA/PyData NYC,Andrea & Brett,New Haven,USA,2015-11-14,2015-11-14,NaN,NaN
145,2015,US Thanksgiving,Andrea & Brett,Meridian,USA,2015-11-23,2015-11-30,NaN,NaN
146,2015,US Thanksgiving,Andrea & Brett,Boise,USA,2015-11-28,2015-11-28,NaN,NaN
144,2015,MLADS,Brett,Redmond,USA,2015-12-06,2015-12-11,NaN,NaN
168,2016,5-year wedding anniversary,Andrea & Brett,Tofino,CAN,2016-02-12,2016-02-16,NaN,NaN
169,2016,5-year wedding anniversary,Andrea & Brett,Ucluelet,CAN,2016-02-15,2016-02-15,NaN,NaN
170,2016,Python Day,Brett,Redmond,USA,2016-04-24,2016-04-30,NaN,NaN


# What cities have we visited in our lives?

In [25]:
import itertools

full_country_names = {
    "CAN": "Canada",
    "USA": "America",
    "GBR": "Great Britain",
    "JPN": "Japan",
    "BEL": "Belgium",
    "CZE": "Czech Republic",
    "ESP": "Spain",
    "FRA": "France",
    "CHE": "Switzerland",
    "CUB": "Cuba",
    "ARG": "Argentina",
    "DOM": "Dominican Republic",
}

def grouper(iterable, n, fillvalue=None):
    """Collect data into fixed-length chunks or blocks.
    
    From https://docs.python.org/3/library/itertools.html#itertools-recipes.
    """
    args = [iter(iterable)] * n
    return itertools.zip_longest(*args, fillvalue=fillvalue)

def column_print(all_names):
    all_names = list(all_names)
    for group in grouper(all_names, 4):
        print('  ', '  '.join(name.ljust(max(len(name) for name in all_names)) for name in group if name is not None))

locations = travelogue[['city', 'country']].drop_duplicates()
countries = [full_country_names[name] for name in locations['country'].drop_duplicates().tolist()]
city_names = tuple(map(', '.join, ((row[0], row[1]) for row in locations.values.tolist()))) 

print("(Everything is listed in chronological order by first visit from left-to-right/top-to-bottom)")
print()
print("{} countries:".format(len(countries)))
column_print(countries)
print()
print("{} cities:".format(len(city_names)))
column_print(city_names)

(Everything is listed in chronological order by first visit from left-to-right/top-to-bottom)

12 countries:
   America             Canada              Great Britain       Japan             
   Belgium             Czech Republic      Spain               France            
   Switzerland         Cuba                Argentina           Dominican Republic

99 cities:
   Anaheim, USA           Yosemite Valley, USA   Sackville, CAN         Summerside, CAN      
   Yarmouth, CAN          Annapolis Royal, CAN   Amherst, CAN           Digby, CAN           
   Kentville, CAN         Moncton, CAN           Shediac, CAN           London, GBR          
   Brighton, GBR          York, GBR              Edinburgh, GBR         Oxford, GBR          
   Chicago, USA           Mammoth Lakes, USA     Washington, D.C., USA  Las Vegas, USA       
   Montclair, USA         New York, USA          Atlantic City, USA     Calgary, CAN         
   Edmonton, CAN          Ely, GBR               Shoreham-by-Sea, GBR

# Is the data valid?
Manually entering data into a spreadsheet is always error-prone, so some quick checks are always useful to catch common mistakes.

In [23]:
import datetime
import re
import urllib.parse

# CITIES
# Latitude
if any(lat < -90 or lat > 90 for lat in cities['latitude']):
    raise ValueError('malformed latitude')
    
# Longitude
if any(lng < -180 or lng > 180 for lng in cities['longitude']):
    raise ValueError('malformed longitude')

# TRAVELOGUE
# Year
if (travelogue['year'] < 1995).any() or (travelogue['year'] > datetime.date.today().year).any():
    raise ValueError('year not between 1995 and today')

# Travellers
if any(x not in colours for x in travelogue['travellers']):
    raise ValueError('unrecognized travellers:', unrecognized)
    
# City
if any(pandas.isnull(travelogue['city'])):
    raise ValueError('missing the city in the travelogue data')
elif not all(travelogue["city"].isin(cities["city"].values)):
    raise ValueError("city in travelogue but not in cities")

# Country
if any(len(x) != 3 or x.upper() != x for x in travelogue['country']):
    bad_countries = []
    raise ValueError('malformed country')
elif not all(travelogue["country"].isin(cities["country"].values)):
    raise ValueError("country in travelogue but not in cities")
elif len(pandas.merge(travelogue, cities, on=["city", "country"])) != len(travelogue):
    raise ValueError("city/country in travelogue not in cities data")
    
# Arrived/Departed
if any(x.arrived > x.departed for x in travelogue.itertuples()):
    raise ValueError("arrival date passed departure date")
    
# Photos
photo_urls = travelogue['photos'].dropna(how='any')
for url in photo_urls:
    parsed_url = urllib.parse.urlparse(url)
    if not parsed_url.scheme or not parsed_url.netloc or not parsed_url.path:
        raise ValueError('malformed photo URL:', url)
        
print('All travelogue data is valid!')

All travelogue data is valid!


In [ ]:
colours = {"Andrea & Brett": "#00FA21", "Andrea": "#F2FA00", "Brett": "#005CFA"}